In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings("ignore")

In [2]:
dataset1=pd.read_csv("prep.csv",index_col=None)
df2=dataset1
df2 = pd.get_dummies(df2, drop_first=True)

In [3]:
df2

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [4]:
indep_X=df2.drop('classification_yes', 1)
dep_Y=df2['classification_yes']

In [5]:
def rfeFeature(indep_X,dep_Y,n):
        rfelist=[]        
        RF = RandomForestRegressor(n_estimators = 10)
        rfemodellist=[RF] 
        for i in  rfemodellist:
            print(i)
            rfe = RFE(i, n)
            rfe_fit = rfe.fit(indep_X, dep_Y)
            rfe_feature=rfe_fit.transform(indep_X)
            rfelist.append(rfe_feature)
            selected_features=indep_X.columns[rfe_fit.support_]
            print("Selected Features",selected_features)
        return rfelist

In [6]:
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)        
        return X_train, X_test, y_train, y_test,sc

In [7]:
def R2_prediction(regressor,X_test,y_test):
        y_pred = regressor.predict(X_test)
        from sklearn.metrics import r2_score
        r_score=r2_score(y_test,y_pred)        
        return  r_score

In [8]:
def random(X_train,y_train,X_test):
        from sklearn.ensemble import RandomForestRegressor
        regressor = RandomForestRegressor(n_estimators = 10)
        regressor.fit(X_train, y_train)
        R2_score=R2_prediction(regressor,X_test,y_test)
        return regressor,R2_score,X_test,y_test

In [15]:
def rfe_regressor(accrf): 
    
    rfedataframe=pd.DataFrame(index=['Linear','SVR','Random','DecisionTree'],columns=['Random'])

    for index in rfedataframe.index:
        rfedataframe['Random'][index]=accrf
    return rfedataframe

In [20]:
rfelist=rfeFeature(indep_X,dep_Y,8) 

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)
Selected Features Index(['al', 'bgr', 'sc', 'sod', 'hrmo', 'pcv', 'sg_c', 'sg_d'], dtype='object')


In [39]:
df2['sg_d']

0      0
1      0
2      0
3      1
4      0
      ..
394    0
395    0
396    0
397    0
398    0
Name: sg_d, Length: 399, dtype: uint8

In [21]:
accrf=[]

In [22]:
for i in rfelist:   
    
    X_train, X_test, y_train, y_test,sc=split_scalar(i,dep_Y)   
            
    regressor,R2_score,X_test,y_test=random(X_train,y_train,X_test)
    accrf.append(R2_score)
    
result=rfe_regressor(accrf[0])

In [23]:
result

,Random
Linear,0.957031
SVR,0.957031
Random,0.957031
DecisionTree,0.957031


In [24]:
import pickle

In [25]:
filename = 'Best_RFE_regression_Model.sav'

In [27]:
pickle.dump(regressor,open(filename,'wb'))

In [28]:
filenamesc = 'sc.pkl'

In [29]:
pickle.dump(sc,open(filenamesc,'wb'))

In [30]:
sc = pickle.load(open('sc.pkl','rb'))

In [40]:
user_input = sc.transform([[3.0,150,7.0,140.0,9.8,36,0,0]])
user_input

array([[ 1.75526219,  0.02303703,  0.65515092,  0.24368447, -0.98589729,
        -0.35577306, -0.52186246, -0.46416285]])

In [41]:
loaded_model = pickle.load(open('Best_RFE_regression_Model.sav','rb'))
Prediction = loaded_model.predict(user_input)

In [42]:
print("Predicted Output :",Prediction)

Predicted Output : [1.]
